In [2]:
import gymnasium as gym 
import pygame
from gymnasium import Env
from gymnasium.spaces import Discrete, Box, Dict, Tuple, MultiBinary, MultiDiscrete 
import numpy as np
import random, time
import os
#from stable_baseline3.stable_baselines3.ppo import ppo
from stable_baselines3 import PPO, DQN
from stable_baselines3.common.env_checker import check_env
from stable_baselines3.common.vec_env import VecFrameStack
from stable_baselines3.common.evaluation import evaluate_policy
#VISITED = [255, 255, 255]
#UNVISITED = [255, 255, 0]
#PLAYER = [255, 0, 0]
print('done')

done


In [4]:
class MowerEnv(Env):
    def __init__(self, len, render_mode = True):
        self.render_mode = render_mode
        self.len = len
        if render_mode == True:
            pygame.init()
            self.screen = pygame.display.set_mode((720, 720))      
        # Actions we can up down left right
        self.action_space = Discrete(4)
        # 0=visited, 1=notvisited
        #gym.spaces.space.Space.seed(1)

        
        self.observation_space = Dict({'grid': Box(low=0, high=255, shape=(self.len,self.len,3), dtype=np.uint8), 'loc': MultiDiscrete([self.len, self.len]), 'area': Discrete(self.len*self.len),'bounds': MultiDiscrete([self.len, self.len, self.len, self.len])})
        
        # Set length
        self.direct = [[-1, 0], [0, 1], [1, 0], [0, -1]]
        
    def step(self, action):
        reward = 0
        arr = self.state['grid']
        pos = self.state['loc']
        # Apply action (no walls for now)
        x = pos[0] + self.direct[action][0]
        y = pos[1] + self.direct[action][1]
        if x < 0 or x >= self.len or y < 0 or y >= self.len:
            reward = -100.0
            self.running_length -= 1 
        else:
            if arr[x][y][0] == 255:
                self.state['area'] -= 1
                reward = 5.0
            elif arr[x][y][0] == 0:
                reward = 0.1
                self.running_length -= 1 
            arr[pos[0]][pos[1]] = 0
            pos[0] = x
            pos[1] = y

        # Reduce shower length by 1 second
        
        # Check if shower is done
        if self.running_length <= 0: 
            done = True
        else:
            done = False
        if self.finished():
            reward = self.len * self.len * 6
            done = True
        # Set placeholder for info
        info = {}
        
        # Return step information
        return (self.state, reward, done, False, info)
    
    def isValid(self, x, y):
        if x < 0 or x < 0 or x >= self.len or y < 0 or y >= self.len:
            return 0 #return False
        
    
    def leastDist(self, x, y):
        ret = 999999999
        arr = self.state['grid']
        for i in range(self.len):
            for j in range(self.len):
                if arr[i][j][0] == 255:
                    ret = min(ret, abs(i - x) + abs(j - y))
        return ret

    def finished(self):
        arr = self.state['grid']
        for i in range(self.len):
            for j in range(self.len):
                if arr[i][j][0] == 255:
                    return False
        return True
    
    def render(self):
        # Implement viz
        if self.render_mode == True:
            pygame.event.get()
            gap = 720//self.len
            arr = self.state['grid']
            pos = self.state['loc']
            for i in range(0, 720, gap):
                for j in range(0, 720, gap):
                    if arr[i//gap][j//gap][0] == 0:
                        color = (255, 255, 255)
                    elif arr[i//gap][j//gap][0] == 255:
                        color = (255, 255,0)
                    if i//gap == pos[0] and j//gap == pos[1]:
                        color = (255, 0, 0)
                    pygame.draw.rect(self.screen, color, (j, i, gap, gap))
            pygame.display.update()
    
    
    def reset(self, seed=None, options=None):
        # Set starting state
        self.state = self.observation_space.sample()
        image = self.state['grid']
        position = self.state['loc']
        bounds = self.state['bounds']
        for i in range(self.len):
            for j in range(self.len):
                image[i][j] = 255
        image[0][0] = 0
        position[0] = position[1] = 0
        self.state['area'] = (self.len * self.len) - 1
        # Set length
        self.running_length = (self.len)*(self.len)//2
        bounds[0] = bounds[1] = 0
        bounds[2] = bounds[3] = self.len-1
        return self.state, {}
    
    def close(self):
        pass

In [5]:
env = MowerEnv(36, True)
env.reset()

(OrderedDict([('area', 1295),
              ('bounds', array([ 0,  0, 35, 35], dtype=int64)),
              ('grid',
               array([[[  0,   0,   0],
                       [255, 255, 255],
                       [255, 255, 255],
                       ...,
                       [255, 255, 255],
                       [255, 255, 255],
                       [255, 255, 255]],
               
                      [[255, 255, 255],
                       [255, 255, 255],
                       [255, 255, 255],
                       ...,
                       [255, 255, 255],
                       [255, 255, 255],
                       [255, 255, 255]],
               
                      [[255, 255, 255],
                       [255, 255, 255],
                       [255, 255, 255],
                       ...,
                       [255, 255, 255],
                       [255, 255, 255],
                       [255, 255, 255]],
               
                      ...,
 

In [4]:
check_env(env)

In [7]:
model = DQN("MultiInputPolicy", env, gamma = 0.999,buffer_size=400000, verbose=1, tau = 0.1, batch_size = 64, exploration_fraction=0.3, exploration_final_eps=0.1, seed=112, _init_setup_model = True, policy_kwargs=dict(net_arch=[128, 128]))

Using cuda device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
Wrapping the env in a VecTransposeImage.
yea boyyyyyyyyyyyyyyyyys
yea boyyyyyyyyyyyyyyyyys


In [25]:
model = PPO("MultiInputPolicy", env, gamma=0.999, ent_coef=0.0001,verbose = 1, seed = 112, policy_kwargs=dict(net_arch=[128, 128]))

Using cuda device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
Wrapping the env in a VecTransposeImage.
yea boyyyyyyyyyyyyyyyyys


In [8]:
model.learn(total_timesteps=1000000,  reset_num_timesteps=True)

----------------------------------
| rollout/            |          |
|    ep_len_mean      | 941      |
|    ep_rew_mean      | -2.1e+03 |
|    exploration_rate | 0.989    |
| time/               |          |
|    episodes         | 4        |
|    fps              | 7584     |
|    time_elapsed     | 0        |
|    total_timesteps  | 3763     |
----------------------------------
-----------------------------------
| rollout/            |           |
|    ep_len_mean      | 917       |
|    ep_rew_mean      | -2.38e+03 |
|    exploration_rate | 0.978     |
| time/               |           |
|    episodes         | 8         |
|    fps              | 7712      |
|    time_elapsed     | 0         |
|    total_timesteps  | 7339      |
-----------------------------------
-----------------------------------
| rollout/            |           |
|    ep_len_mean      | 890       |
|    ep_rew_mean      | -2.58e+03 |
|    exploration_rate | 0.968     |
| time/               |           |
|  

In [19]:
print(Box(low=0, high=255, shape=(66,66,3)).shape)

(66, 66, 3)


In [11]:
average = 0
length = 0
for i in range(2):
    obs, _ = env.reset()
    while True:
        # Next action:
        # (feed the observation to your agent here)
        action = model.predict(obs)

        # Processing:
        obs, reward, terminated, _, info = env.step(action[0])
        average += reward
        length += 1
        # Rendering the game:
        # (remove this two lines during training)
        env.render()
        #time.sleep(1 / 60)  # FPS
        
        # Checking if the player is still alive
        if terminated:
            break

    env.close()
print(length/5)
print(average/5)

435.6
847.8599999999803
